<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/Fama_%2B_LSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Setting

In [ ]:
'''
종목개수 50개 
기대 CAGR 25%이상

소형주 500개
PER/PBR/PCR 순위 매김
통합 순위 작성
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)
print(company.shape)
company.head(10)

In [ ]:

from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import html
from urllib import request as rq
import requests

In [ ]:
LS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'


In [ ]:
sample_code = '005930'

In [ ]:
snap_url = rq.urlopen(SNAP_URL%sample_code).read()
snap_soup = BeautifulSoup(snap_url, 'html.parser')


In [ ]:
mv_cells= snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td', {'class': "r"})
mv = float(mv_cells[-4].string.replace(',',''))
mv

#Get PCR

In [ ]:
ls_html = rq.urlopen(LS_URL%sample_code).read()
ls_soup = BeautifulSoup(ls_html, 'html.parser')


In [ ]:
pcr_ls= ls_soup.find('tr', {'id':'p_grid1_10'}).find_all('td', {'class': "r"})
pcr = float(pcr_ls[3].string.replace(',',''))
pcr

#Get PER

In [ ]:
per_ls = ls_soup.find('tr', {'id':'p_grid1_9'}).find_all('td', {'class': "r"})
per = float(per_ls[3].string.replace(',',''))
per

# Get PBR

In [ ]:
pbr_ls = ls_soup.find('tr', {'id':'p_grid1_12'}).find_all('td', {'class': "r"})
pbr = float(pbr_ls[3].string.replace(',',''))
pbr

In [ ]:
code_list = company['종목코드']
sample_code_list = code_list[1000:1050]
sample_code_list


#Create Crawling Fucntion

In [ ]:
def crawler(li):
    LS_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'
    SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'

    result = {}

    for code in tqdm(li):
        try:
            # get company name 
            name = company[company['종목코드'] == code]['회사명'].values[0]
            ls_html = rq.urlopen(LS_URL%code).read()
            ls_soup = BeautifulSoup(ls_html, 'html.parser')
            snap_url = rq.urlopen(SNAP_URL%sample_code).read()
            snap_soup = BeautifulSoup(snap_url, 'html.parser')

            # get Market_Value
            mv_cells= snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td', {'class': "r"})
            mv = float(mv_cells[-4].string.replace(',',''))
            
            #get PER
            per_ls = ls_soup.find('tr', {'id':'p_grid1_9'}).find_all('td', {'class': "r"})
            per = float(per_ls[3].string.replace(',',''))

            #get PBR
            pbr_ls = ls_soup.find('tr', {'id':'p_grid1_12'}).find_all('td', {'class': "r"})
            pbr = float(pbr_ls[3].string.replace(',',''))

            #get PCR
            pcr_ls = ls_soup.find('tr', {'id':'p_grid1_10'}).find_all('td', {'class': "r"})
            pcr = float(pcr_ls[3].string.replace(',',''))
            
            
            # Insert into result dict
            result[name] = [code, mv, per, pbr, pcr]

        except(TypeError, AttributeError, IndexError, ValueError) :
            pass

    # convert dict into DataFrame
    result_df = pd.DataFrame(result)

    # transposing DataFrame
    result_df = result_df.transpose()

    # set Column name
    result_df.columns = ['Code','Market_Value', 'PER', 'PBR', 'PCR']

    return result_df

In [ ]:
result_df = crawler(sample_code_list)
tmp_df = result_df.copy()
tmp_df.head(10)

#Filtering Small Cap

#Scoring

In [ ]:
top_range =  int(len(tmp_df) * 0.3)


middle_range =  int(len(tmp_df) * 0.7)
print(middle_range, top_range)

In [ ]:
tmp_df['Code'].isin(tmp_df.sort_values(by='PER')[:top_range]['Code'])

In [ ]:

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PER')[:top_range]['Code']),'PER_Score'] = 3

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PER')[top_range:middle_range]['Code']),'PER_Score'] = 2

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PER')[middle_range:]['Code']),'PER_Score'] = 1


In [ ]:
tmp_df

In [ ]:

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PBR')[:top_range]['Code']),'PBR_Score'] = 3

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PBR')[top_range:middle_range]['Code']),'PBR_Score'] = 2

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PBR')[middle_range:]['Code']),'PBR_Score'] = 1


In [ ]:
tmp_df

In [ ]:

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PCR')[:top_range]['Code']),'PCR_Score'] = 3

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PCR')[top_range:middle_range]['Code']),'PCR_Score'] = 2

tmp_df.loc[tmp_df['Code'].isin(tmp_df.sort_values(by='PCR')[middle_range:]['Code']),'PCR_Score'] = 1


In [ ]:
tmp_df

In [ ]:
tmp_df['Total_Score'] = tmp_df['PER_Score'] + tmp_df['PBR_Score'] + tmp_df['PCR_Score']


In [ ]:
max_point_df = tmp_df[tmp_df['Total_Score'] == 9.0]
max_point_df

In [ ]:
max_point_df.sort_values(by='Total_Score', ascending=False).head(50)


In [ ]:
final_df = max_point_df.sort_values(by='PCR', ascending=False).head(50)
final_df

In [ ]:
# csv파일로 내보내기
final_df.to_csv('/content/drive/MyDrive/Colab Notebooks/result.csv')